In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
os.getcwd()

'/home/kaiz/Desktop/Gender-Recognition-MLOps'

#### Create DataIngestionConfig class to hold the configuration details about data ingestion 

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnGenderClassifier.constants import *
from cnnGenderClassifier.utils.common import read_yaml, create_directories

In [7]:
print(CONFIG_FILE_PATH)
print(PARAMS_FILE_PATH)

config/config.yaml
params.yaml


#### Create ConfigurationManager

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # create method to get data ingestion related configurations
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

#### Let's Create DataIngestion class

In [9]:
import os
import urllib.request as request
import zipfile
import gdown
from cnnGenderClassifier.utils import logger
from cnnGenderClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # download file from the location
    def download_file(self) -> str:
        """Fetch data from the URL
        """

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to the file {zip_download_dir}")

            file_id = dataset_url.split("/")[5]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    # extract the zipfile
    def extract_zip_file(self):
        """
        zip file path: str
        Extracts the zip file into the data directory
        Function returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### Create DataIngestion pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-27 16:36:38,262: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-27 16:36:38,267: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-27 16:36:38,270: INFO: common: created directory at artifacts]
[2024-04-27 16:36:38,272: INFO: common: created directory at artifacts/data_ingestion]
[2024-04-27 16:36:38,274: INFO: 3837693123: Downloading data from https://drive.google.com/file/d/1poPH6O8LWq7I5Aw0i6XUBSgXEl_AWvzw/view?usp=sharing to the file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1poPH6O8LWq7I5Aw0i6XUBSgXEl_AWvzw
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1poPH6O8LWq7I5Aw0i6XUBSgXEl_AWvzw&confirm=t&uuid=17c05551-1d98-4731-b243-6031790dd40c
To: /home/kaiz/Desktop/Gender-Recognition-MLOps/artifacts/data_ingestion/data.zip
100%|██████████| 227M/227M [02:38<00:00, 1.43MB/s] 

[2024-04-27 16:39:21,196: INFO: 3837693123: Downloaded data from https://drive.google.com/file/d/1poPH6O8LWq7I5Aw0i6XUBSgXEl_AWvzw/view?usp=sharing into file artifacts/data_ingestion/data.zip]
